# Algorithm Examples in Python vs PySpark
The objective of this Notebook is to help students understand the differences between implementing simple algorithms in Python and PySpark. 

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext("local")
spark = SparkSession.builder.getOrCreate()

# Sum of elements in a list

In [2]:
# Python code
arr = [1, 2, 3, 4, 5]
result = 0
for i in range(len(arr)):
    result += arr[i]
print(result)

15


In [3]:
# PySpark code
rdd = sc.parallelize([1, 2, 3, 4, 5])
result = rdd.reduce(lambda a, b: a + b)
print(result)

15


# Word Count

In [4]:
# Python code
text = "hello world hello"
word_dict = {}
for word in text.split():
    word_dict[word] = word_dict.get(word, 0) + 1
print(word_dict)


{'hello': 2, 'world': 1}


In [5]:
# PySpark code
text = sc.parallelize(["hello world hello"])
result = (text.flatMap(lambda line: line.split(" "))
          .map(lambda word: (word, 1))
          .reduceByKey(lambda a, b: a + b)
          .collect())

print(dict(result))

{'hello': 2, 'world': 1}


# Finding Maximum Element

In [6]:
# Python code
arr = [2, 4, 1, 8, 5]
max_val = arr[0]
for i in range(1, len(arr)):
    if arr[i] > max_val:
        max_val = arr[i]
print(max_val)

8


In [7]:
# PySpark code
rdd = sc.parallelize([2, 4, 1, 8, 5])
result = rdd.reduce(lambda a, b: a if a > b else b)
print(result)

8


# Counting Occurrences of a Specific Element

In [8]:
# Python code
arr = [2, 3, 4, 2, 8, 2]
target = 2

result = 0
for i in range(len(arr)):
    if arr[i] == target:
        result += 1
print(result)

3


In [9]:
# PySpark code
rdd = sc.parallelize([2, 3, 4, 2, 8, 2])
target = 2
result = rdd.filter(lambda x: x == target).count()
print(result)

3


# Finding Prime Numbers in a Range

In [10]:
# Python code

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5)+1):
        if n % i == 0:
            return False
    return True

prime_numbers = []
for i in range(2, 30):
    if is_prime(i):
        prime_numbers.append(i)
print(prime_numbers)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


In [11]:
# PySpark code

rdd = sc.parallelize(range(2, 30))
prime_numbers = rdd.filter(is_prime).collect()
print(prime_numbers)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


# Histogram Generation

In [12]:
arr = [1, 1, 1, 2, 2, 3]
result = {}
for i in range(len(arr)):
    result[arr[i]] = result.get(arr[i], 0) + 1
print(result)

{1: 3, 2: 2, 3: 1}


In [13]:
rdd = sc.parallelize([1, 1, 1, 2, 2, 3])
result = rdd.countByValue()
print(result)

defaultdict(<class 'int'>, {1: 3, 2: 2, 3: 1})


# Sum of Squares

In [14]:
# Python code
arr = [1, 2, 3, 4, 5]
sum_of_squares = 0
for i in range(len(arr)):
    sum_of_squares += arr[i] ** 2
print(sum_of_squares)

55


In [15]:
# PySpark code
rdd = sc.parallelize([1, 2, 3, 4, 5])
sum_of_squares = rdd.map(lambda x: x ** 2).reduce(lambda a, b: a + b)
print(sum_of_squares)

55


# TF-IDF (Term Frequency-Inverse Document Frequency)

TF-IDF stands for Term Frequency-Inverse Document Frequency. It is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. This method is widely used in information retrieval and text mining.

**Term Frequency (TF)**

Term Frequency measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (the total number of terms in the document) as a way of normalization:

$$
TF(t) = \left(\frac{\text{Number of times term } t \text{ appears in a document}}{\text{Total number of terms in the document}}\right)
$$

**Inverse Document Frequency (IDF)**

Inverse Document Frequency measures how important a term is. While computing TF, all terms are considered equally important. However, certain terms, like "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scaling up the rare ones, by computing the following:

$$
IDF(t) = \log\left(\frac{1 + \text{Total number of documents}}{1 + \text{Number of documents with term } t}\right) + 1
$$

This formula ensures that terms with very low frequency do not get a zero IDF by adding 1 in the numerator and denominator. The logarithmic scale is used to ensure that the IDF doesn't grow too quickly with the increase in the number of documents.

**TF-IDF Calculation**

TF-IDF is simply the product of TF and IDF:

$$
TFIDF(t, d) = TF(t, d) \times IDF(t)
$$

This value is higher when a term is more frequent in a specific document but less frequent across all documents, which implies the term is quite significant in the particular document.

**Application in Text Mining**

TF-IDF has a variety of applications, mainly in systems involving natural language processing (NLP) and information retrieval such as:

- **Search engines**: Ranking documents based on query terms.
- **Document clustering**: Grouping similar documents.
- **Text summarization**: Extracting key terms that reflect the most relevant information in documents.
- **Feature extraction**: Transforming textual data into a format suitable for machine learning algorithms.

In [16]:
# Python code
from collections import Counter
import math

documents = ["apple orange apple", "apple lemon", "orange lemon"]
N = len(documents)
idf_dict = Counter()
tf_dict_list = []

for doc in documents:
    tf_dict = Counter(doc.split())
    tf_dict_list.append(tf_dict)
    for word in tf_dict.keys():
        idf_dict[word] += 1
        
for word in idf_dict.keys():
    idf_dict[word] = math.log(N / idf_dict[word])
    
tfidf_documents = []

for tf_dict in tf_dict_list:
    tfidf = {}
    for word, count in tf_dict.items():
        tfidf[word] = count * idf_dict[word]
    tfidf_documents.append(tfidf)

print(tfidf_documents)

[{'apple': 0.8109302162163288, 'orange': 0.4054651081081644}, {'apple': 0.4054651081081644, 'lemon': 0.4054651081081644}, {'orange': 0.4054651081081644, 'lemon': 0.4054651081081644}]


In [17]:
# PySpark code
documents = ["apple orange apple", "apple lemon", "orange lemon"]
rdd = sc.parallelize(documents)

# TF calculation
tf_rdd = rdd.map(lambda doc: Counter(doc.split()))

# IDF calculation
idf_rdd = tf_rdd.flatMap(lambda tf: tf.keys()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

N = rdd.count()
idf_values = idf_rdd.mapValues(lambda count: math.log(N / count)).collectAsMap()

# TF-IDF calculation
tfidf_rdd = tf_rdd.map(lambda tf: {word: tf[word] * idf_values[word] for word in tf})

tfidf = tfidf_rdd.collect()
print(tfidf)

[{'apple': 0.8109302162163288, 'orange': 0.4054651081081644}, {'apple': 0.4054651081081644, 'lemon': 0.4054651081081644}, {'orange': 0.4054651081081644, 'lemon': 0.4054651081081644}]
